# Recurrent Residual Convolutional Neural Network

### Simon Laurent Lebailly, 2549365, s9sileba@teams.uni-saarland.de
### Christian Mathieu Schmidt, 2537621, s9cmscmi@teams.uni-saarland.de

## Import libaries

In [ ]:
import torch
import torchvision
import numpy as np

## Use CUDA if possible

In [ ]:
#Check if CUDA is available, if not use the CPU.
train_on_GPU = torch.cuda.is_available()
device = torch.device('cuda' if train_on_GPU else 'cpu')

if train_on_GPU:
    print('CUDA available!')
else:
    print('CUDA not available!')

## Define Hyperparameters

In [ ]:
batch_size = 4
learning_rate = 0.0001

## Import dataset Cityscapes

In [ ]:
from torch.utils import data
from torchvision import transforms
from torchvision import datasets


#Define transformation for dataset
transform = transforms.Compose(
        [
            transforms.Resize((256,512)),
            transforms.CenterCrop(256, 512),
            transforms.ToTensor(),
            transforms.Normalize(
                mean = [0.485, 0.456, 0.406],
                std = [0.229, 0.224, 0.225]
            ),
        ]
    )

root_path = 'data'

#Import dataset for training
train_set = datasets.Cityscapes(root_path, split='train', mode='fine', transform=transform)
quantity_train = len(train_set)
print('Quantity training data: '+ str(quantity_train))

#Prepare training dataset for NN
train_loader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)


#Import dataset for validation
validation_set = datasets.Cityscapes(root_path, split='val', mode='fine', transform=transform)
quantity_validation = len(validation_set)
print('Quantity validation data: '+ str(quantity_validation))

#Prepare validation dataset for NN
validation_loader = data.DataLoader(validation_set, batch_size=batch_size, shuffle=True)


#Import dataset for testing
test_set = datasets.Cityscapes(root_path, split='test', mode='fine', transform=transform)
quantity_test = len(test_set)
print('Quantity testing data: '+ str(quantity_test))

#Prepare test dataset for NN
test_loader = data.DataLoader(test_set, batch_size=batch_size, shuffle=False)